# Chapter 1 — Linear Systems and Matrices (Colab Companion)

This Colab notebook provides Python utilities and worked demos for core concepts from Chapter 1:

- Translating linear systems to **augmented matrices**  
- **Elementary row operations**, **row-echelon (REF)** and **reduced row-echelon (RREF)**
- **Gaussian / Gauss–Jordan elimination** with optional **partial pivoting**
- Reading off **pivot** vs **free** variables; **consistency** test (pivot in last column)
- **Vandermonde interpolation** example

> Tip: All code uses **exact arithmetic** (rationals) via SymPy by default, which keeps steps clean and reproducible.

In [ ]:
# Setup (SymPy + NumPy). In Colab these are usually available already.
# If needed, uncomment the next line to install/upgrade.
# !pip -q install sympy numpy

import sympy as sp
import numpy as np

sp.init_printing(use_unicode=True)  # Pretty printing of matrices and rationals

## 1) Ref/Rref helpers (exact arithmetic)

Utilities to compute REF and RREF with SymPy.

In [ ]:
# --- REF / RREF helpers ---

def to_matrix(A):
    """
    Ensure input is a SymPy Matrix with Rational entries when possible.
    Accepts: Python lists, NumPy arrays, or SymPy Matrix.
    """
    if isinstance(A, sp.MatrixBase):
        M = A.copy()
    else:
        M = sp.Matrix(A)
    # Try to coerce to rationals for exact arithmetic
    def _coerce(x):
        if isinstance(x, (int, np.integer)):
            return sp.Rational(int(x))
        try:
            return sp.nsimplify(x)
        except Exception:
            return x
    return M.applyfunc(_coerce)

def compute_ref_rref(A):
    """
    Return (REF, RREF, pivot_cols) for the given matrix A.
    Uses exact arithmetic where possible.
    """
    M = to_matrix(A)
    REF = M.echelon_form()  # Not fully reduced
    RREF, pivots = M.rref() # Fully reduced
    return REF, RREF, pivots

# Demo
if __name__ == "__main__":
    A = [[1, 2, -1, -4],
         [2, 3, -1, -11],
         [-2, 0, -3, 22]]
    REF, RREF, piv = compute_ref_rref(A)
    print("Input A:"); sp.pprint(to_matrix(A))
    print("\nRow Echelon Form (REF):"); sp.pprint(REF)
    print("\nReduced Row Echelon Form (RREF):"); sp.pprint(RREF)
    print("Pivot columns (0-indexed):", piv)

## 2) System → Augmented matrix (with pretty display)

Turn symbolic equations into the augmented matrix `[A | b]`.

In [ ]:
# --- System to augmented matrix ---

def system_to_augmented(equations, variables):
    """
    Parameters
    ----------
    equations : list of sympy.Eq or strings like "2*x + y - z = 1"
    variables : list/tuple of sympy symbols in the desired order, e.g. (x, y, z)

    Returns
    -------
    A : SymPy Matrix (coefficients)
    b : SymPy Matrix (rhs column vector)
    Ab: SymPy Matrix (augmented [A | b])
    """
    # Parse equations if given as strings
    eqs = []
    for e in equations:
        if isinstance(e, str):
            left, right = e.split("=")
            eqs.append(sp.Eq(sp.sympify(left), sp.sympify(right)))
        else:
            eqs.append(sp.Eq(sp.sympify(e.lhs), sp.sympify(e.rhs)))
    A, b = sp.linear_eq_to_matrix(eqs, variables)
    A = to_matrix(A)
    b = to_matrix(b)
    Ab = A.row_join(b)
    return A, b, Ab

# Demo
if __name__ == "__main__":
    x1, x2, x3 = sp.symbols('x1 x2 x3')
    eqs = ["x1 + 3*x2 - 2*x3 = -5",
           "4*x1 + 9*x2 + 1*x3 = -2",
           "-6*x2 + 2*x3 = 24"]
    A, b, Ab = system_to_augmented(eqs, (x1, x2, x3))
    print("A ="); sp.pprint(A)
    print("b ="); sp.pprint(b)
    print("[A | b] ="); sp.pprint(Ab)

## 3) Gauss–Jordan with step-by-step row operations

Produce RREF *and* a readable log of each elementary row operation.
Optional **partial pivoting** (default: `True`) improves numerical stability if you switch to floats.

In [ ]:
# --- Gauss–Jordan with steps ---

def rref_with_steps(A, partial_pivoting=True):
    """
    Compute RREF while recording each elementary row operation.

    Parameters
    ----------
    A : array-like
        Matrix or augmented matrix.
    partial_pivoting : bool
        If True, swap in the row with max |entry| in the pivot column.

    Returns
    -------
    R : SymPy Matrix
        The reduced row-echelon form.
    steps : list of dict
        Each dict has 'op' (str) and 'matrix' (SymPy Matrix snapshot).
    """
    M = to_matrix(A)
    M = M.applyfunc(sp.nsimplify)  # normalize
    m, n = M.shape
    i = 0  # current pivot row
    j = 0  # current pivot col
    steps = []

    def log(op):
        steps.append({"op": op, "matrix": M.copy()})

    while i < m and j < n:
        # 1) Choose pivot row
        pivot_row = None
        if partial_pivoting:
            # pick row with max absolute value in column j (from i..m-1)
            col_vals = [(r, abs(M[r, j])) for r in range(i, m)]
            r_best, val_best = max(col_vals, key=lambda t: t[1])
            if val_best != 0:
                pivot_row = r_best
        else:
            for r in range(i, m):
                if M[r, j] != 0:
                    pivot_row = r
                    break

        if pivot_row is None:  # no pivot in this column
            j += 1
            continue

        # 2) Swap to the top (row i)
        if pivot_row != i:
            M.row_swap(i, pivot_row)
            log(f"Swap R{i+1} ↔ R{pivot_row+1}")

        # 3) Scale pivot row to make pivot = 1
        pivot = M[i, j]
        if pivot != 1:
            M.row_op(i, lambda val, _: val / pivot)
            log(f"R{i+1} := (1/{pivot}) · R{i+1}")

        # 4) Eliminate all other entries in column j
        for r in range(m):
            if r != i and M[r, j] != 0:
                factor = M[r, j]
                M.row_op(r, lambda val, k: val - factor * M[i, k])
                log(f"R{r+1} := R{r+1} - ({factor}) · R{i+1}")

        i += 1
        j += 1

    # Final snapshot
    log("RREF reached")
    return M, steps

def print_steps(steps):
    """Pretty-print a steps log produced by rref_with_steps."""
    for idx, s in enumerate(steps, 1):
        print(f"Step {idx}: {s['op']}")
        sp.pprint(s["matrix"])
        print("-" * 40)

# Demo on an example matrix
if __name__ == "__main__":
    A = [[0, 2, 12, 11, -20],
         [2, 8, 16, 0, 12],
         [1, 3, 2, 3, -1]]
    R, steps = rref_with_steps(A, partial_pivoting=True)
    print_steps(steps)

## 4) Consistency & solution structure from an augmented matrix

Given an augmented matrix `[A | b]`, check **consistency** (pivot in last column) and list **pivot** vs **free** variables.

In [ ]:
# --- Augmented-matrix analysis ---

def analyze_augmented(Ab, num_vars=None):
    """
    Inspect an augmented matrix [A | b] in RREF to report:
    - consistency (pivot in last column?)
    - pivot columns and free columns (for A-part)
    - rank
    """
    M = to_matrix(Ab)
    R, piv = M.rref()
    m, n = R.shape
    if num_vars is None:
        num_vars = n - 1  # assume one RHS
    last_col = n - 1
    consistent = True
    # row that is [0 ... 0 | nonzero] indicates inconsistency
    for r in range(m):
        if all(R[r, c] == 0 for c in range(num_vars)) and R[r, last_col] != 0:
            consistent = False
            break
    pivot_cols = [p for p in piv if p < num_vars]
    free_cols  = [c for c in range(num_vars) if c not in pivot_cols]
    return {"consistent": consistent,
            "rank": len(pivot_cols),
            "pivot_cols": pivot_cols,
            "free_cols": free_cols}, R

# Demo
if __name__ == "__main__":
    Ab = [[1, 3, -2, 0, 5],
          [0, 1,  4, -1, 2],
          [0, 0,  0,  1, -3]]
    report, R = analyze_augmented(Ab)
    print("RREF:"); sp.pprint(R)
    print("Report:", report)

## 5) Vandermonde interpolation

Build the Vandermonde matrix \(V\) from points \((x_i, y_i)\) and solve for the coefficients of the degree \(n-1\) polynomial.

In [ ]:
# --- Vandermonde interpolation ---

def vandermonde_fit(xs, ys):
    """
    Given points (xs, ys) with distinct xs, fit p(x)=a_{n-1}x^{n-1}+...+a1 x + a0.
    Returns (coeff_vector, polynomial).
    """
    xs = list(xs); ys = list(ys)
    n = len(xs)
    V = sp.Matrix([[sp.Integer(x)**k for k in range(n-1, -1, -1)] for x in xs])
    y = sp.Matrix(ys)
    # Solve V * a = y using exact arithmetic when possible
    a = V.LUsolve(y)  # works over rationals as well
    x = sp.symbols('x')
    # Build polynomial with ascending powers for readability
    poly = sum(a[n-1-k]*x**k for k in range(n))
    return a, sp.expand(poly)

# Demo
if __name__ == "__main__":
    xs = [0, 1, 2]
    ys = [1, 2, 5]
    a, p = vandermonde_fit(xs, ys)
    print("Coefficients (a_{2}, a_{1}, a_{0}) ="); sp.pprint(a.T)
    print("p(x) ="); sp.pprint(p)

## 6) Quick utilities

Shortcuts to solve small systems and to convert solutions into a parametric form when there are free variables.

In [ ]:
# --- Solve small systems and parametrize ---

def solve_augmented(Ab, var_symbols):
    """
    Solve [A | b] as a linear system for given var_symbols using SymPy.
    Returns a dict or parametric solution (SymPy form).
    """
    Ab = to_matrix(Ab)
    nvars = len(var_symbols)
    A = Ab[:, :nvars]
    b = Ab[:, nvars:]
    solset = sp.linsolve((A, b))
    return list(solset)[0]  # tuple (possibly parametric)

# Demo
if __name__ == "__main__":
    x1, x2, x3 = sp.symbols('x1 x2 x3')
    Ab = [[1, 0, -2, 3],
          [0, 1,  1, -1],
          [0, 0,  0,  0]]
    sol = solve_augmented(Ab, (x1, x2, x3))
    print("Solution (possibly parametric):", sol)

---

### How to use this notebook

1. Run the **Setup** cell.  
2. Use **`compute_ref_rref`** to get REF/RREF and pivots.  
3. Use **`system_to_augmented`** to translate symbolic equations to `[A | b]`.  
4. Use **`rref_with_steps`** + **`print_steps`** to see each row operation.  
5. Use **`analyze_augmented`** to check consistency and identify free variables.  
6. Try **`vandermonde_fit`** for polynomial interpolation tasks.